In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('C:\\Users\\tomdx\\Documents\\find hotel')

In [2]:
df = pd.read_csv('Campaign_Data.csv')

In [3]:
df['date'] = pd.to_datetime(df['date'])
df['clicks'] = df['clicks'].fillna(0)
df['total_costs'] = df['total_costs'].fillna(0)
df = df.drop_duplicates()


In [4]:
df.duplicated().sum()

0

In [5]:
df.head()

,week,market_id,date,device,clicks,destination_country_name,bookings,total_costs,commision,matchtype,destination_super_region,key_adgroup_id
0,46-2015,18,2015-12-11,Desktop,1.0,NaN,0,0.63,0.0,exact,NaN,19607539
1,41-2015,18,2015-04-10,Desktop,0.0,NaN,0,0.00,0.0,phrase,NaN,19326908
2,6-2016,18,2016-02-02,Tablet,1.0,NaN,0,0.93,0.0,phrase,NaN,19326908
3,7-2016,18,2016-10-02,Desktop,1.0,NaN,0,0.67,0.0,phrase,NaN,19326908
4,45-2015,18,2015-04-11,Mobile,0.0,NaN,0,0.00,0.0,phrase,NaN,19326910


In [6]:
jf = df.groupby(['destination_country_name','device'])['clicks','bookings','commision','total_costs'].sum().reset_index()
jf.sort_values('commision',ascending=False).head()

<ipython-input-6-690592b9ba1f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  jf = df.groupby(['destination_country_name','device'])['clicks','bookings','commision','total_costs'].sum().reset_index()


,destination_country_name,device,clicks,bookings,commision,total_costs
192,Kiribati,Mobile,1.0,0,3627.000000,0.08
128,Germany,Desktop,0.0,192,2624.083815,0.00
176,Italy,Desktop,1133.0,27,1321.863175,623.81
391,United States,Desktop,937.0,24,1033.569195,702.99
341,Spain,Desktop,491.0,7,472.342230,300.80


In [7]:
jf['commison_per-booking'] = jf['commision']/jf['bookings']
jf['profit'] = jf['commision'] - jf['total_costs']
jf['profit_per_booking'] = jf['profit']/jf['bookings']
jf['cpc'] = jf['total_costs']/jf['clicks']
 
jf['cr'] = jf['bookings']/jf['clicks']
jf['roi'] = round((jf['profit']/jf['total_costs'])*100,2)

In [8]:
jf.loc[jf['roi']<0,'profitable'] = 'No'
jf.loc[jf['roi']>0,'profitable'] = 'Yes'
jf.loc[jf['roi']==0,'profitable'] = 'break-even'

In [9]:
jf = jf.replace([np.inf, -np.inf], np.nan)


In [10]:
jf['click_range'] = pd.cut(x=jf['clicks'], bins=[-1,0,100, 200, 300, 400,500,600,700,800,900,1000,1500],\
       labels=[0,'1-100','101-200','201-300','301-400','401-500','501-600','601-700','701-800','801-900','901-1000','1000+']) 

In [11]:
jf['bookings_range'] = pd.cut(x=jf['bookings'], bins=[-1,0,5, 10, 15, 20,25,30,35],\
       labels=[0,'1-5','6-10','11-15','16-20','21-25','26-30','31+']) 

jf['break_even_cpc'] = jf['profit_per_booking'] * jf['cr'] 
jf['ideal_cpc'] = jf['break_even_cpc'] * 0.7
jf['change'] = (jf['break_even_cpc'] - jf['cpc'])/jf['cpc']